In [1]:
# This is a practice Artificial Neural Network
# The problem being solved is based off of a model bank with fake data
# The bank has customers that have left for whatever reason
# The goal is to find why these customers have left using information such as account balance and gender
# The last column of the data states whether or not the customer has left the bank
import math
import time

import numpy as np #Math operations library
import matplotlib.pyplot as plt #Visualization library
import pandas as pd #Matrix handler

import keras # Brings in tensorflow with it
from keras.models import Sequential # Used for initialization of ANN
from keras.layers import Dense, Conv3D, MaxPooling3D, Flatten, Dropout, BatchNormalization# adds layers to ANN
from keras.wrappers.scikit_learn import KerasClassifier # ability to turn network into a function definition
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler #Methods to change categorical strings to numbers and scaling ability
from sklearn.model_selection import train_test_split, GridSearchCV # Splits data into training and testing
from sklearn.metrics import confusion_matrix # Creates truth table for evaluating results

parameters = {'batch_size': 100,
              'epochs': 100,
              'learning_rate': 0.00001,
              'nodes': 2400,
              'hidden_layers': 2
             } # Creates list of parameters to test to find most successful one
parameters['optimizer'] = Adam(parameters['learning_rate'])

class data_handler:
    def __init__(self, pkl_location):
        self.data = pd.read_pickle(pkl_location)
        self.data = self.data.sample(frac=1).reset_index(drop=True)
        print(self.data)
        self.x = self.data[list(range(0, 264))]# max 264 x columns
        self.y = self.data[['Rest', 'Emote', 'Solve']]
        print(self.data)
        del self.data
        
        self.tenth = math.floor(self.x.shape[0] / 10)
        
    def get_cross_val(self, iteration):
        if iteration > 10:
            raise ValueError('Crossval Iteration Exceeds 10')
        
        iteration = iteration - 1
        train_array = list(range(0, iteration*self.tenth)) + list(range((iteration+1)*self.tenth, self.x.shape[0]))
        test_array = list(range(iteration*self.tenth, (iteration+1)*self.tenth))
        
        x_train = self.x.iloc[train_array]
        y_train = self.y.iloc[train_array]
        x_test = self.x.iloc[test_array]
        y_test = self.y.iloc[test_array]
        
        return x_train, y_train, x_test, y_test

def build_classifier(parameters):
    classifier = Sequential() # This is the ANN object
    classifier.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
    
    for layer in range(0, parameters['hidden_layers']):
        classifier.add(Dense(units=parameters['nodes'], kernel_initializer='uniform', activation='relu')) #Creates first hidden layer
        classifier.add(Dropout(rate=0.5))
        classifier.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

    classifier.add(Dense(units=3, kernel_initializer='uniform', activation='softmax')) # Output layer. Only 1 ouput category, sigmoid activation to get probability of sureness
    # Note: Softmax applies to a dependent variable that has more than 2 categories
    # i.e. fMRI categorizations
    
    classifier.compile(loss='categorical_crossentropy',
              optimizer=parameters['optimizer'],
              metrics=['accuracy'])
    # Notes
    # adam is a kind of stochastic gradient descent
    # For multivariabel, use categorical cross entropy
    # Accuracy is predefined
    return classifier
# Creates a standard Keras type classifier composed of the defined network for
# k-means testing
data = data_handler('./atlas.pkl')

Using TensorFlow backend.


                0         1         2         3          4         5  \
0       -0.432690 -1.297000 -1.062600 -0.659030   0.749150  0.668930   
1        0.048841  0.017907  0.063133  0.066374  -0.021142  0.054850   
2        0.018148 -0.002884 -0.112950  0.066071   0.000690 -0.025123   
3        0.077216  0.155570  0.148840 -0.078220  -0.046353  0.056544   
4       -0.007275  0.001259 -0.003566 -0.039139  -0.029627 -0.021410   
5       -0.032138 -0.080774 -0.080089  0.067738   0.010799 -0.014069   
6       -0.094736 -0.100570 -0.071752 -0.052315   0.023642  0.151830   
7       -0.142300  0.056452 -0.138510  0.037676  -0.169830  0.056670   
8       -0.059631 -0.074320 -0.124400 -0.075413   0.024633  0.027707   
9       -0.050301 -0.047367  0.090624 -0.002637  -0.038612 -0.012719   
10      -0.129250 -0.162840 -0.019609 -0.038128  -0.009866  0.014628   
11       0.012858  0.018466 -0.103660  0.052701   0.001016 -0.044332   
12      -0.149640 -0.129800  0.146140 -0.118890  -0.055036 -0.21

KeyError: ('Rest', 'Emote', 'Solve')

In [ ]:
epoch = 0
tensorboard = TensorBoard(log_dir="../logs/{}".format(time.time()))

for e in range(parameters['epochs']):
    X_batch, y_batch = batcher.next_batch(parameters['batch_size'])
    classifier.fit(np.array(X_batch), np.array(y_batch), batch_size=parameters['batch_size'], epochs=1,callbacks=[tensorboard], validation_split=0.3)
    print(e)

Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 632s 45s/step - loss: 1.0707 - acc: 0.5000 - val_loss: 1.0961 - val_acc: 0.5000
0
Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 632s 45s/step - loss: 1.1203 - acc: 0.3571 - val_loss: 1.0784 - val_acc: 0.5000
1
Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 632s 45s/step - loss: 1.1008 - acc: 0.3571 - val_loss: 1.0815 - val_acc: 0.6667
2
Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 633s 45s/step - loss: 1.0929 - acc: 0.4286 - val_loss: 1.0957 - val_acc: 0.3333
3
Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 633s 45s/step - loss: 1.1005 - acc: 0.2143 - val_loss: 1.0926 - val_acc: 0.3333
4
Train on 14 samples, validate on 6 samples
Epoch 1/1
14/14 [==============================] - 633s 45s/step - loss: 1.10

In [20]:
for i in range(0, 10):
    i = i + 1
    tensorboard = Tensorboard(log_dir="../logs/{}".format(time.time()))
    x_train, y_train, x_test, y_test = data.get_cross_val

1
2
3
4
5
6
7
8
9
10
